# Gaussian Process Regression for WiFi Fingerprinting-based Localization

In this assignment, you will work with Gaussian process for regression. Recall in Assignment 1, we use a given path loss model and trilateration to determine locations using WiFi radio signal strenght (RSS) from different access points (AP). Training the path loss model itself can be done using polynomial regression. Indoor environments are generally quite complex and cannot be characterized using a single path loss model accurately. An alternative approach is to treat RSS vectors at each location as a *fingerprint*. If these fingerprints are unique, we can use them to determine locations.   

As an example, in the following figure, there are four APs. We use colors to represent the amplitude of RSS values (hot colors indicate higher values). As one moves away from an AP, the RSS decreases. However, the contour curves of equal RSS readings from one AP are often not circular (and thus the path loss model fails). 

![WiFi Fingerprint](RSS.jpg)

For simplicity, we only consider **2D localization**. The most naive approach is to build a lookup table of fingerprints and locations from training data. During testing, given the RSS measurements, one can search for the  location that gives the best match (by the nearest neighbor search). However, such a solution requires dense measurements across the target space since we do not know what the measurements are at locations not in the training set. Additionally, the naive lookup approach fails to account of uncertainty in RSS measurements that is prevalent in indoor environments. 

For an AP $i$, we can model the *mean* RSS value (in dBm) at location $(x_1, x_2)$ from AP $i$ as a function $f_i: R^2 \rightarrow R$. In the training phase, we conduct site survey and measure RSS values at a number of known locations. The training data $\{X, Y\}$ thus consists of $\{(x_1^j, x_2^j), (RSS_1^j, RSS_2^j, \ldots, RSS_m^j), j=1, 2, \ldots, N\}$, where $RSS_i^j$ is the RSS at $j$th location from AP $i$. Note that the number of visible APs tends to differ from one location to another. 

We assume $f_i$'s are drawn from $GP(m(x), k(x,x'))$, where $m(x)$ is the mean process and $k(x,x')$ is the kernel. With GP regression, we obtain the predictive model for RSS values from AP $i$ at an unknown location(s) $X_*$ by,
$$
f_i(X_*)| X, y, X_* \sim N(\bar{f_i}^*, cov(f_i^*)), 
$$
where $\bar{f_i}^* = K(X_*, X)[K(X,X) + \sigma_n^2I]^{-1}y_i$ and $cov(f_i^*) = K(X_*, X_*) - K(X_*, X)[K(X,X)+\sigma_n^2 I]^{-1}K(X,X_*)$

To determine the location a device given its RSS measurements $[RSS_{i_1}, RSS_{i_2}, \ldots, RSS_{i_l}]$, one uses the maximum a posteriori (MAP) estimation. Formally, 
$$
\arg\max_{X_*} \sum_{c = 1}^{l} \log f_{i_c}(X_*)| X, y, X_*
$$

One way to solve this optimization problem is to divide the search space into regular grids and find the grid point that maximizes the objective function. 

<br>
<font color="blue">
[Reference] 
Ferris, B, Hahnel, D & Fox, D 2007, Gaussian processes for signal strength-based location estimation. in Robotics: Science and Systems. vol. 2, MIT Press Journals, pp. 303-310, 2nd International Conference on Robotics Science and Systems, RSS 2006, Philadelphia, United States, 16-19 August.</font>

In [6]:
import ianus
import numpy as np
import matplotlib.pyplot as plt
import GPy 
import math 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import scipy

GPy package can be found at https://sheffieldml.github.io/GPy/ <br>
**Before you start the assignment, please go through the lecture slides and the GP demo discussed in class**

## 1. Read WiFi and Location Data
WiFi, AP and location data are similar to Assignment 1 with one minor difference. In AP.csv, only the mac address of APs are given. There are no locations of the APs. 

In [7]:
datapath = './dataset/'

X_raw, apinfo = ianus.read_wifi_ap_data(datapath+'wifi.pbf', datapath+'AP.csv')
Y = ianus.read_location_data(datapath + 'location.pbf')

mac_address = list(apinfo.keys())
num_AP = len(mac_address)
num_loc = Y.shape[1]

assert (num_loc == len(X_raw))

# Search space
min_X, min_Y, max_X, max_Y = [np.amin(Y[0]), np.amin(Y[1]), np.amax(Y[0]), np.max(Y[1])]

# Mesh for localization. Find a location among NxN grids points that maximizes a posteriori distribution
N = 5
mesh = np.meshgrid(np.linspace(min_X, max_X, N), np.linspace(min_Y, max_Y, N))
grid = np.array([mesh[0].reshape(N*N,1), mesh[1].reshape(N*N,1)]).squeeze().T

## 2. Gaussian Process Regression (Cross validation)
Step 1. Parition the data into training and testing sets randomly (e.g., 90% of the locations and their RSS readings are in the training set and the remaining 10% in the testing set). <br>
Step 2. Train a GP for **each** feature (AP) using RBF kernel (maximally, there are num_AP GP's). <br>
Step 3. Optimize the hyper-parameters via optimization *model.optimize*

**Note: the training and testing set division is among locations NOT measurements from the same locations**

In [8]:
X_mean = np.zeros((num_AP, num_loc))
for i in range(num_loc):
    num_scans = X_raw[i].shape[1]
    X_mean[:,i] = np.nanmean(X_raw[i][:,0:num_scans],axis=1)
X_mean = X_mean.T
Y_new = Y.T[:,0:2]
error_sum = 0
X = Y_new
y = X_mean

kf = KFold(n_splits=10)
kf.get_n_splits(X) 
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = y[train_index], y[test_index]

    GP = []
    GP_index = []
    for i in range(num_AP):
        X_AP = X_train[~np.isnan(Y_train[:,i])]
        Y_AP = Y_train[:,i]
        Y_AP = Y_AP[~np.isnan(Y_AP)]
        Y_AP = np.reshape(Y_AP,(Y_AP.shape[0],1))

        if(X_AP.shape[0] > 10):
            kernel = GPy.kern.RBF(input_dim=2,lengthscale =10)
            m = GPy.models.GPRegression(X_AP,Y_AP,kernel)
            m.optimize(messages=False,max_f_eval = 1000)
            GP.append(m)
            GP_index.append(i)

    GP_num = len(GP)
    mean = np.zeros([grid.shape[0],GP_num])
    var = np.zeros([grid.shape[0],GP_num])
    for i in range(GP_num):
        result = GP[i].predict(grid)
        mean[:,i] = result[0].reshape((grid.shape[0],))
        var[:,i] = result[1].reshape((grid.shape[0],))

    num_test = X_test.shape[0]
    Y_hat = np.zeros([num_test,2])
    print(num_test)
    for num in range(num_test):
        sum = np.zeros(grid.shape[0])
        for p in range(grid.shape[0]):
            for k in range(GP_num):
                RSS = Y_test[num,GP_index[k]]
                if (~np.isnan(RSS)):
                    sum[p] += scipy.stats.multivariate_normal.logpdf(RSS,mean[p,k],math.sqrt(var[p,k]))
        max_index = np.argmax(sum)
        Y_hat[num] = grid[max_index]

    Y_testing = X_test.T
    Y_hat = Y_hat.T
    errors = np.linalg.norm(Y_testing - Y_hat, axis=0)
    print("Mean localization error = %f"%np.mean(errors))
    error_sum += np.mean(errors)
cross_validation_error = error_sum/10
print("error by 10-folds validation = %f"%cross_validation_error)

 C:\Users\wenqi\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning:Mean of empty slice


(511, 2)
(511, 70)


 C:\Users\wenqi\Anaconda3\lib\site-packages\GPy\kern\src\rbf.py:43: RuntimeWarning:overflow encountered in square


52
[ 16.2   0.5]
[ 0.   0.5]
[ 0.   0.5]
[ 16.2   0.5]
[ 0.   0.5]
[ 16.2   0.5]
[ 16.2   0.5]
[ 0.   0.5]
[ 0.   0.5]
[ 16.2   0.5]
[ 16.2   0.5]
[ 16.2   0.5]
[ 16.2   0.5]
[ 16.2   0.5]
[ 16.2   0.5]
[ 16.2   0.5]
[ 16.2   0.5]
[ 16.2   0.5]
[ 16.2   0.5]
[ 16.2   0.5]
[ 16.2   0.5]
[ 16.2   0.5]
[ 16.2   0.5]
[ 32.4    13.705]
[ 32.4    13.705]
[ 32.4   0.5]
[ 32.4   0.5]
[ 32.4   0.5]
[ 32.4   0.5]
[ 32.4   0.5]
[ 32.4   0.5]
[ 32.4   0.5]
[ 32.4   0.5]
[ 32.4   0.5]
[ 32.4   0.5]
[ 32.4   0.5]
[ 32.4   0.5]
[ 32.4   0.5]
[ 32.4   0.5]
[ 48.6   0.5]
[ 32.4   0.5]
[ 48.6   0.5]
[ 48.6   0.5]
[ 48.6   0.5]
[ 48.6   0.5]
[ 48.6   0.5]
[ 48.6   0.5]
[ 48.6   0.5]
[ 48.6   0.5]
[ 48.6   0.5]
[ 48.6   0.5]
[ 48.6   0.5]
Mean localization error = 4.930310
51
[ 48.6   0.5]
[ 64.8   0.5]
[ 64.8   0.5]
[ 64.8   0.5]
[ 64.8   0.5]
[ 64.8   0.5]
[ 64.8   0.5]
[ 64.8   0.5]
[ 64.8   0.5]
[ 64.8   0.5]
[ 64.8   0.5]
[ 64.8   0.5]
[ 64.8   0.5]
[ 64.8   0.5]
[ 64.8   0.5]
[ 0.   0.5]
[ 0.   0.5]

 C:\Users\wenqi\Anaconda3\lib\site-packages\GPy\kern\src\stationary.py:167: RuntimeWarning:overflow encountered in true_divide
 C:\Users\wenqi\Anaconda3\lib\site-packages\GPy\kern\src\rbf.py:46: RuntimeWarning:invalid value encountered in multiply


51
[ 32.4   0.5]
[ 32.4   0.5]
[ 32.4   0.5]
[ 32.4   0.5]
[ 32.4   0.5]
[ 32.4   0.5]
[ 32.4   0.5]
[ 32.4   0.5]
[ 32.4   0.5]
[ 32.4   0.5]
[ 32.4   0.5]
[ 32.4   0.5]
[ 48.6   0.5]
[ 32.4   0.5]
[ 32.4   0.5]
[ 48.6   0.5]
[ 48.6   0.5]
[ 48.6   0.5]
[ 48.6   0.5]
[ 48.6   0.5]
[ 48.6   0.5]
[ 48.6   0.5]
[ 48.6   0.5]
[ 48.6   0.5]
[ 48.6   0.5]
[ 48.6   0.5]
[ 48.6   0.5]
[ 48.6   0.5]
[ 64.8   0.5]
[ 64.8   0.5]
[ 64.8   0.5]
[ 64.8   0.5]
[ 64.8   0.5]
[ 64.8   0.5]
[ 64.8   0.5]
[ 64.8   0.5]
[ 64.8   0.5]
[ 64.8   0.5]
[ 64.8   0.5]
[ 32.4    13.705]
[ 48.6    13.705]
[ 48.6    13.705]
[ 48.6   0.5]
[ 48.6    13.705]
[ 48.6    13.705]
[ 48.6    13.705]
[ 48.6    13.705]
[ 48.6    13.705]
[ 48.6    13.705]
[ 48.6    13.705]
[ 48.6    13.705]
Mean localization error = 4.932045
51
[ 48.6    13.705]
[ 48.6    13.705]
[ 48.6    13.705]
[ 48.6    13.705]
[ 48.6    13.705]
[ 48.6   26.91]
[ 48.6    13.705]
[ 48.6    13.705]
[ 48.6    13.705]
[ 48.6    13.705]
[ 48.6   26.91]
[ 48.6 